## Create Edits that match post edit class distribution of target class to real edits

In [1]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from datetime import datetime

In [2]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, list_to_dict, ensure_dir, informal_log, write_lists
from utils.df_utils import load_and_preprocess_csv
from utils.visualizations import histogram
from utils.model_utils import prepare_device, quick_predict
from parse_config import ConfigParser
from test import predict, predict_with_bump
import datasets.datasets as module_data
import model.model as module_arch
import model.metric as module_metric
import model.loss as module_loss
from trainer.editor import Editor

In [3]:
# Define constants, paths
class_list_path = os.path.join('metadata', 'cinic-10', 'class_names.txt')

config_path = 'configs/copies/edit_experiments/cinic10_imagenet_bump_noise.json'

In [4]:
# Load config file, models, and dataloader
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)

config_dict = read_json(config_path)
config = ConfigParser(config_dict)
device, device_ids = prepare_device(config_dict['n_gpu'])

In [5]:
# Load model
layernum = config.config['layernum']
model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
model.eval()


CIFAR10PretrainedModelEdit(
  (model): VGG(
    (normalize): InputNormalize()
    (layer0): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer3): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, moment

In [9]:
# Load datasets
# config = ConfigParser(config_dict)
data_loader_args = dict(config.config["data_loader"]["args"])
dataset_args = dict(config["dataset_args"])

val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Obtain loss function and metric functions
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

In [6]:
target_class_name = 'airplane'
target_class_idx = class_idx_dict[target_class_name]
n_select = 100
timestamp = '0127_103716'
save_timestamp = '0208_112555'

root_dir = os.path.join('saved', 'edit', 'trials', 'CINIC10_ImageNet-VGG_16', '{}' + '_{}'.format(n_select), timestamp)
csv_path_template = os.path.join(root_dir, 'results_table.csv')


### Get the Target Class Distribution Across All Edits for Specific Class

In [33]:
def save_class_distribution(csv_path, 
                            target_class_idx,
                            target_class_name,
                            show=False,
                            histogram_save_path=None,
                            data_save_path=None):
    
    df = load_and_preprocess_csv(
        csv_path=csv_path,
        drop_duplicates=['ID']
    )
    
    # Obtain number of predictions for target class pre edit
    pre_edit_class_distribution = df['Pre Class Dist'].to_numpy()
    pre_edit_class_distribution = np.stack(pre_edit_class_distribution)
    target_pre_edit_class_predictions = np.mean(pre_edit_class_distribution[:, target_class_idx])
    
    # Obtain number of predictions for target class post edit for each trial
    class_distribution = df['Post Class Dist'].to_numpy()
    class_distribution = np.stack(class_distribution, axis=0)
    target_class_distribution = class_distribution[:, target_class_idx]
    # target_class_bins = np.bincount(target_class_distribution)
    if histogram_save_path is None:
        histogram_save_path = os.path.join(os.path.dirname(csv_path), 'graphs', 'summary', 'target_class_distribution.png')
    title = 'Post Edit {} Class Distribution for {} Edits'.format(target_class_name, target_class_name)
    xlabel = 'Num. {} Predictions Post Edit'.format(target_class_name)
    ylabel = 'Num. Edits'
    
    bin_values, bins, _ = histogram(
        data=target_class_distribution,
        n_bins=50,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        marker=target_pre_edit_class_predictions,
        show=show,
        save_path=histogram_save_path)
    
    bin_tuples = []
    for bin_idx in range(len(bins) - 1):
        bin_tuples.append((bins[bin_idx], bins[bin_idx+1]))

    save_data = {
        "n_target_predictions": target_class_distribution,
        "histogram_bin_values": bin_values,
        "histogram_bins": bins
    }
    
    if data_save_path is None:
        data_save_path = os.path.join(os.path.dirname(csv_path), 'target_class_distribution.pth')
        torch.save(save_data, data_save_path)
    
    print("Saved target class distribution & histogram data to {}".format(data_save_path))
    
    plt.close('all')
    return save_data

#### Loop over all classes and save histograms and distributions

In [ ]:
for class_name in class_list:
    csv_path = csv_path_template.format(class_name)
    save_class_distribution(
        csv_path=csv_path,
        target_class_idx=class_idx_dict[class_name],
        target_class_name=class_name,
        show=False)
    

### For a specific class, make bump edits that match the distribution of target class prediction

In [22]:
# def predict_with_bump(data_loader,
#                       model,
#                       target_class_idx,
#                       bump_amount,
#                       loss_fn,
#                       metric_fns,
#                       device,
#                       save_path=None):
#     '''
#     Run the model on the data_loader, calculate metrics, and log

#     Arg(s):
#         data_loader : torch Dataloader
#             data to test on
#         model : torch.nn.Module
#             model to run
#         loss_fn : module
#             loss function
#         metric_fns : list[model.metric modules]
#             list of metric functions
#         device : torch.device
#         save_path : str or None
#             if not None, save metrics to save_path

#     Returns :
#         log : dict{} of metrics
#     '''

#     # Hold data for calculating metrics
#     outputs = []
#     targets = []

#     # Ensure model is in eval mode
#     if model.training:
#         model.eval()

#     with torch.no_grad():
#         for idx, item in enumerate(tqdm(data_loader)):
#             if len(item) == 3:
#                 data, target, path = item
#             else:
#                 data, target = item
#             data, target = data.to(device), target.to(device)
#             output = model(data)

#             # Store outputs and targets
#             outputs.append(output)
#             targets.append(target)

#     # Concatenate predictions and targets
#     outputs = torch.cat(outputs, dim=0)
#     targets = torch.cat(targets, dim=0)
    
#     # Adjust output softmax by bump amount
#     outputs[:, target_class_idx] += bump_amount

#     # Calculate loss
#     loss = loss_fn(outputs, targets).item()
#     n_samples = len(data_loader.sampler)
#     log = {'loss': loss}

#     # Calculate predictions based on argmax
#     predictions = torch.argmax(outputs, dim=1)

#     # Move predictions and target to cpu and convert to numpy to calculate metrics
#     predictions = predictions.cpu().numpy()
#     targets = targets.cpu().numpy()

#     # Calculate metrics
#     log = module_metric.compute_metrics(
#         metric_fns=metric_fns,
#         prediction=predictions,
#         target=targets)

#     if save_path is not None:
#         ensure_dir(os.path.dirname(save_path))
#         torch.save(log, save_path)

#     return log

In [7]:
def match_bump_edits(data_loader,
                     model,
                     loss_fn,
                     metric_fns,
                     device,
                     bin_lows,
                     target_class_idx,
                     save_dir,
                     debug=True):
    '''
    Get bins
    for each bin, get the mean
    until the target class predictions on val match the mean,
        adjust the target
    store (mean, bump amount)
    '''
    # save_dir = os.path.dirname(str(config.save_dir))
    informal_log_path = os.path.join(save_dir, 'informal_log.txt')
    bump_dictionary = {}
    
    n_bins = len(bin_lows) - 1
    
    n_bins_left = n_bins
    min_bin = bin_lows[0]
    bin_width = bin_lows[1] - bin_lows[0]
    
    bin_means = []
    bump_amount = 0.2
    delta = 1.25
    
    bumps = [-1 for i in range(n_bins)]
    class_predictions = [-1 for i in range(n_bins)]
    logs = [None for i in range(n_bins)]
    
    cur_bump_idx = 0
    cur_itr = 0
    delta_reset = 1.25
    
    # Obtain pre edit metrics
    pre_edit_log = predict(
        data_loader=data_loader,
        model=model,
        loss_fn=loss_fn,
        metric_fns=metric_fns,
        device=device,
        save_path=None)

    while cur_bump_idx < n_bins:
        cur_itr += 1
        if delta <= 1 or delta > 2:
            delta = delta_reset
        if debug:
            informal_log("[{}]***NEW ITR {}***".format(
                datetime.now().strftime(r'%m%d_%H%M%S'), cur_itr), informal_log_path)

        if bumps[cur_bump_idx] > -1:
            bump_amount = bumps[cur_bump_idx]
            cur_bump_idx += 1
            
            continue
        
        log = predict_with_bump(
            data_loader=data_loader,
            model=model,
            loss_fn=loss_fn,
            metric_fns=metric_fns,
            device=device,
            target_class_idx=target_class_idx,
            bump_amount=bump_amount,
            save_path=None)
        
        # Obtain num. predictions for target class and determine bin idx
        post_class_distribution = log['predicted_class_distribution']
        target_class_predictions = post_class_distribution[target_class_idx]
        bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
        
        # Check for out of bounds/already visited bin index
        while bin_idx < cur_bump_idx:
            bump_amount *= delta
            if debug:
                informal_log("bin_idx < cur_bump_idx ({} and {}) delta={} new bump_amount: {}".format(
                    bin_idx, cur_bump_idx, delta, bump_amount), informal_log_path)
                
            log = predict_with_bump(
                data_loader=data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=bump_amount,
                save_path=None)

            # Obtain num. predictions for target class and determine bin idx
            post_class_distribution = log['predicted_class_distribution']
            target_class_predictions = post_class_distribution[target_class_idx]
            bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
                
            
            # continue
        if bin_idx >= n_bins:
            if cur_bump_idx == 0:
                bump_amount /= delta
            else:
                bump_amount = (bump_amount + bumps[cur_bump_idx - 1]) / 2
            informal_log("bin_idx > n_bins ({}) delta={} new bump_amount: {}".format(
                bin_idx, delta, bump_amount), informal_log_path)
            continue
            
        # Sanity check correct bin_idx
        assert target_class_predictions >= bin_lows[bin_idx] and \
               target_class_predictions <= bin_lows[bin_idx+1]
        
        # If bump value for this bin yet, assign it
        if bumps[bin_idx] == -1:
            bumps[bin_idx] = bump_amount
            class_predictions[bin_idx] = target_class_predictions
            logs[bin_idx] = log
            if debug:
                informal_log("assigned bin_idx {}".format(bin_idx), informal_log_path)
                informal_log("Cur bump_idx: {} bump amount: {} target_class_pred: {} bin_idx: {}".format(
                    cur_bump_idx, bump_amount, target_class_predictions, bin_idx), informal_log_path)
        
        # Check if this is our current bin we're trying to fill
        if bin_idx == cur_bump_idx:
            cur_bump_idx += 1
            bump_amount *= delta
            if debug:
                informal_log("updating cur_bump_idx to {}, bump_amount to {}"
                             .format(cur_bump_idx, bump_amount), informal_log_path)
            
        # If we're not at the current cell,
        else:
            # Decrease bump amount by half
            if cur_bump_idx == 0:
                bump_amount /= 2
            # Set bump amount to something in the middle between current and last successful
            else:
                bump_amount = (bump_amount + bumps[cur_bump_idx-1]) / 2 
            if debug:
                informal_log("Overshot to bin {}. Decreasing bump amount to {} aiming for bin {}".format(
                    bin_idx, bump_amount, cur_bump_idx), informal_log_path)
        informal_log("bumps: {}".format(bumps), informal_log_path)

    
    
    save_data = {
        'pre_edit_metrics': pre_edit_log,
        'bump_amounts': bumps,
        'target_class_predictions': class_predictions,
        'metrics': logs
    }
    save_path = os.path.join(save_dir, 'bumps_preds_metrics.pth'.format())
    torch.save(save_data, save_path)
    if debug:
        informal_log("Saved data to {}".format(save_path), informal_log_path)
    return save_data
            

#### Run match_bump_edits() to get how much to bump to match each bucket

In [10]:
target_class_distribution_path = os.path.join(root_dir, 'target_class_distribution.pth').format(
    target_class_name)
target_class_distribution = torch.load(target_class_distribution_path)
bin_lows = target_class_distribution['histogram_bins']

bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
ensure_dir(bump_save_dir)
# Save a copy of histogram info to save_dir
torch.save(target_class_distribution, os.path.join(bump_save_dir, 'target_class_distribution.pth'))
# Run bump experiments
print("Obtaining class distribution for {} from {}".format(target_class_name, target_class_distribution_path))
print("Saving results to {}".format(bump_save_dir))
match_bump_edits(
    data_loader=val_paths_data_loader,
    model=model,
    loss_fn=loss_fn,
    metric_fns=metric_fns,
    device=device,
    bin_lows=bin_lows,
    target_class_idx=target_class_idx,
    save_dir=bump_save_dir)

Obtaining class distribution for airplane from saved/edit/trials/CINIC10_ImageNet-VGG_16/airplane_100/0127_103716/target_class_distribution.pth
Saving results to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100


 26%|██████████████████▍                                                   | 72/274 [00:32<01:30,  2.23it/s]


KeyboardInterrupt: 

#### Sanity check that resulting histogram matches original from real edits

In [ ]:
bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
metrics_save_path = os.path.join(bump_save_dir, 'bumps_preds_metrics.pth')

bumped_target_class_dist = torch.load(metrics_save_path)

bumped_hist_data = []


target_class_dist_dict = torch.load(target_class_distribution_path)

for n_target_predictions, bucket_value in zip(
    bumped_target_class_dist['target_class_predictions'], 
    target_class_distribution['histogram_bin_values']):
    cur_data = [n_target_predictions for i in range(int(bucket_value))]
    bumped_hist_data += cur_data
    
bins = target_class_distribution['histogram_bins']
bin_values = target_class_distribution['histogram_bin_values']

histogram_save_path = os.path.join(
    bump_save_dir, 
    'graphs',
    'summary',
    'bumped_target_class_distribution.png')
bump_bin_values, bump_bins, _= histogram(
        data=bumped_hist_data,
        n_bins=bins, #50,
        title='Bumped Post Edit {} Class Distribution to Match {} Edits'.format(target_class_name, target_class_name),
        xlabel='Num. {} Predictions Post Bump'.format(target_class_name),
        ylabel='Num. Edits',
        save_path=histogram_save_path,
        show=True)

assert (bin_values == bump_bin_values).all()

### Perform Bump for a Corresponding Edit

In [21]:
def match_bump(n_target_predictions,
               target_class_idx,
               bumps_preds_metrics,
               results_save_dir,
               data_loader,
               model,
               loss_fn,
               metric_fns,
               device,
               cushion=5,
               debug=True):
    '''
    Given a number of predictions for target class, obtain bump amount to match and save post edit metrics in results_save_dir
    
    Arg(s):
        n_target_predictions : int
            Number of predictions to obtain for target class
        target_class_idx : int
            index of target class
        bumps_preds_metrics : dict
            saved data from match_bump_edits()
        results_save_dir : str
            directory to save results to
        data_loader : torch.utils.data.DataLoader
            validation data loader to obtain metrics for
        model : torch.nn.Module
            model
        loss_fn : module
            loss function
        metric_fns : list[model.metric modules]
            list of metric functions
        device : torch.device
            GPU device to run model on
        cushion : int
            how far away cur_n_target_predictions can be from n_target_predictions on either side to break loop (buffer)
        debug : bool
            control verbosity
        
            
    Returns: 
        None
    '''
    # Unpack bumps_preds_metrics
    pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    target_class_predictions = bumps_preds_metrics['target_class_predictions']
    bump_amounts = bumps_preds_metrics['bump_amounts']
    
    # Ensure that the n_target_predictions > pre edit # predictions of target class
    pre_edit_n_target_predictions = pre_edit_metrics['predicted_class_distribution'][target_class_idx]
    assert pre_edit_n_target_predictions < n_target_predictions, \
        "n_target_predictions ({}) must be greater than pre-edit n_target_predictions ({})".format(n_target_predictions, pre_edit_n_target_predictions) 
    n_predictions_total = np.sum(pre_edit_metrics['predicted_class_distribution'])
    assert n_target_predictions <= n_predictions_total, \
        "n_target_predictions ({}) must be less than total number of data in dataloader ({})".format(n_target_predictions, n_predictions_total)
    # Find index above and below n_target_predictions
    bin_high_idx = -1
    for bin_idx, target_class_prediction in enumerate(target_class_predictions):
        if target_class_prediction > n_target_predictions:
            bin_high_idx = bin_idx
            break
    if bin_high_idx == -1: # Past upper end 
        # n_predictions_upper_bound
        bump_amount_upper_bound = bump_amounts[bin_high_idx] * 2
        bump_amount_lower_bound = bump_amounts[bin_high_idx]
        # raise ValueError("Out of bounds bin index")
        # return
    else: 
        n_predictions_upper_bound = target_class_predictions[bin_high_idx]
        bump_amount_upper_bound = bump_amounts[bin_high_idx]

        # Store lower bounds for bump_amount and n_predictions
        bin_low_idx = bin_high_idx - 1
        if bin_low_idx > -1: # First bin is already higher than n_target_predictions
            n_predictions_lower_bound = target_class_predictions[bin_low_idx]
            bump_amount_lower_bound = bump_amounts[bin_low_idx]
        else:
            n_predictions_lower_bound = 0
            bump_amount_lower_bound = 0
        
    # check to make sure that the targe
    # cur_bump_amount = (bump_amount_lower_bound + bump_amount_upper_bound) / 2
    cur_n_target_predictions = 0
    
    if debug:
        print("target n_predictions: {}".format(n_target_predictions))
        print("Initial bounds for bump: ({}, {})".format(bump_amount_lower_bound, bump_amount_upper_bound))
    while abs(cur_n_target_predictions - n_target_predictions) > cushion:
        cur_bump_amount = (bump_amount_lower_bound + bump_amount_upper_bound) / 2
        log = predict_with_bump(
                data_loader=data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=cur_bump_amount,
                save_path=None)

        # Obtain num. predictions for target class and determine bin idx
        post_class_distribution = log['predicted_class_distribution']
        cur_n_target_predictions = post_class_distribution[target_class_idx]
        if debug:
            print("cur_bump_amount: {}, cur_n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
        if cur_n_target_predictions > n_target_predictions:
            bump_amount_upper_bound = cur_bump_amount
            if debug:
                print("Updated upper bound to {}".format(bump_amount_upper_bound))
        elif cur_n_target_predictions < n_target_predictions:
            bump_amount_lower_bound = cur_bump_amount
            if debug:
                print("Updated lower bound to {}".format(bump_amount_lower_bound))
        
    if debug:
        print("final results: bump amount: {} n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
    return cur_bump_amount, cur_n_target_predictions
    

In [23]:
# testing

bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
metrics_save_path = os.path.join(bump_save_dir, 'bumps_preds_metrics.pth')
bumps_preds_metrics = torch.load(metrics_save_path)

n_target_predictions = 65000
results_save_dir = os.path.join('temp')
match_bump(n_target_predictions=n_target_predictions,
           target_class_idx=target_class_idx,
           bumps_preds_metrics=bumps_preds_metrics,
           results_save_dir=results_save_dir,
           data_loader=val_paths_data_loader,
           model=model,
           loss_fn=loss_fn,
           metric_fns=metric_fns,
           device=device,
           cushion=5,
           debug=True)

target n_predictions: 65000
Initial bounds for bump: (6.507315933965021, 13.014631867930042)


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:12<00:00, 22.66it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


cur_bump_amount: 9.760973900947532, cur_n_target_predictions: 68683
Updated upper bound to 9.760973900947532


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:12<00:00, 21.61it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


cur_bump_amount: 8.134144917456275, cur_n_target_predictions: 66104
Updated upper bound to 8.134144917456275


...

cur_bump_amount: 8.029290705629691, cur_n_target_predictions: 65027
Updated upper bound to 8.029290705629691


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:06<00:00, 43.59it/s]

cur_bump_amount: 8.0277020054505, cur_n_target_predictions: 65005
Updated upper bound to 8.0277020054505
final results: bump amount: 8.0277020054505 n_target_predictions: 65005



/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


(8.0277020054505, 65005)

### Copy results such that we can call src/utils/results_to_csv.py to get csv of results

In [41]:
def replicate_bump_results(bumps_preds_metrics,
                           target_class_distribution,
                           results_save_dir,
                           trial_paths_path):
    '''
    Given how much to bump logit by to match target class distribution, create results for each tiral
    
    Arg(s):
        bumps_preds_metrics : dict
            result of match_bump_edits()
        target_class_distribution : dict
            histogram data, result from save_class_distribution()
        results_save_dir : str
            location of results folder
        trial_paths_path : str
            path to file to store paths to each trial
            
    '''
    pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    
    metrics = bumps_preds_metrics['metrics']
    bin_values = target_class_distribution['histogram_bin_values']
    bins = target_class_distribution['histogram_bins']
    bin_lows = bins[:-1]
    bin_highs = bins[1:]

    trial_paths = []
    for bin_idx, bin_value in enumerate(tqdm(bin_values)):
        n_repeat = int(bin_value)
        if n_repeat == 0:
            continue
            
        # Obtain post edit metrics
        post_edit_metrics = metrics[bin_idx]
        
        for itr in range(n_repeat):
            # Create directory for this trial
            trial_id = 'bin_{}_{}_itr_{}'.format(bin_lows[bin_idx], bin_highs[bin_idx], itr)
            trial_dir = os.path.join(results_save_dir, trial_id)
            
            # Log path to trial_paths.txt
            trial_paths.append(trial_dir)
            
            trial_save_dir = os.path.join(trial_dir, 'models')
            ensure_dir(trial_save_dir)
            # Save pre edit metrics
            torch.save(pre_edit_metrics, os.path.join(trial_save_dir, 'pre_edit_metrics.pth'))
            # Save post edit metrics
            torch.save(post_edit_metrics, os.path.join(trial_save_dir, 'post_edit_metrics.pth'))
        
    return trial_paths

def replicate_bump_noise_results(bumps_preds_metrics,
                                 target_class_distribution,
                                 results_save_dir,
                                 trial_paths_path,
                                 progress_report_path,
                                 # noise_edit=False,
                                 target_class_idx,
                                 config,
                                 data_loader,
                                 loss_fn,
                                 metrics_fns,
                                 editor,
                                 device):
    '''
    Given how much to bump logit by to match target class distribution, create results for each tiral
    
    Arg(s):
        bumps_preds_metrics : dict
            result of match_bump_edits()
        target_class_distribution : dict
            histogram data, result from save_class_distribution()
        results_save_dir : str
            location of results folder
        trial_paths_path : str
            path to file to store paths to each trial
        progress_report_path : str
            path to log progress
        target_class_idx : int
            index of target class
        config : return from ConfigParser
            config object to instantiate model from
        data_loader : torch.utils.data.DataLoader
            validation data loader to obtain metrics for
        loss_fn : module
            loss function
        metric_fns : list[model.metric modules]
            list of metric functions
        editor : Editor (from trainer/editor.py) or None
            pass in an editor if performing random edits
        device : torch.device
            GPU device to run model on
    '''
    pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    
    bump_amounts = bumps_preds_metrics['bump_amounts']
    metrics = bumps_preds_metrics['metrics']
    bin_values = target_class_distribution['histogram_bin_values']
    bins = target_class_distribution['histogram_bins']
    bin_lows = bins[:-1]
    bin_highs = bins[1:]

    trial_paths = []
    
    layernum = config.config['layernum']
    # if noise_edit:
    #     assert target_class_idx is not None
    #     assert config is not None
    #     assert data_loader is not None
    #     assert loss_fn is not None
    #     assert metrics_fns is not None
    #     assert editor is not None
    #     assert device is not None
        
    for bin_idx, bin_value in enumerate(bin_values):
        n_repeat = int(bin_value)
        if n_repeat == 0:
            continue
            
        informal_log("Bin {} [{}, {})".format(bin_idx, bin_lows[bin_idx], bin_highs[bin_idx]), progress_report_path)
        for itr in range(n_repeat):
            informal_log("\tItr {}/{}".format(itr+1, n_repeat), progress_report_path)
            # Create directory for this trial
            trial_id = 'bin_{}_{}_itr_{}'.format(bin_lows[bin_idx], bin_highs[bin_idx], itr)
            trial_dir = os.path.join(results_save_dir, trial_id)

            # Log path to trial_paths.txt
            trial_paths.append(trial_dir)
            informal_log(trial_dir, trial_paths_path)
            # Create save directories, save pre_edit_metrics
            trial_save_dir = os.path.join(trial_dir, 'models')
            pre_edit_metrics_save_path = os.path.join(trial_save_dir, 'pre_edit_metrics.pth')
            post_edit_metrics_save_path = os.path.join(trial_save_dir, 'post_edit_metrics.pth')
            ensure_dir(trial_save_dir)
            # Save pre edit metrics
            torch.save(pre_edit_metrics, pre_edit_metrics_save_path)
            if not noise_edit:
                # Save calculated post edit metrics
                # Obtain and save post edit metrics
                post_edit_metrics = metrics[bin_idx]
                torch.save(post_edit_metrics, post_edit_metrics_save_path)
                
            else:
                # Load in fresh model
                
                model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
                model.eval()
                
                # edit model with noise
                editor.noise_edit(model=model)
                
                # obtain bump amount
                bump_amount = bump_amounts[bin_idx]
                log = predict_with_bump(
                    data_loader=data_loader,
                    model=model,
                    loss_fn=loss_fn,
                    metric_fns=metric_fns,
                    device=device,
                    target_class_idx=target_class_idx,
                    bump_amount=bump_amount,
                    save_path=post_edit_metrics_save_path)
            
                
                
    return trial_paths

In [45]:
bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
ensure_dir(bump_save_dir)

bump_preds_metrics_path = os.path.join(bump_save_dir, 'bumps_preds_metrics.pth')
target_class_distribution_path = os.path.join(bump_save_dir, 'target_class_distribution.pth')

# Load objects
bumps_preds_metrics = torch.load(bump_preds_metrics_path)
target_class_distribution = torch.load(target_class_distribution_path)

# Determine whether to use noise_edits or not
noise_edit = config.config['editor']['noise_edit']
if noise_edit: 
    results_save_dir = os.path.join(bump_save_dir, 'bump_noise', 'results')
    ensure_dir(results_save_dir)
    progress_report_path = os.path.join(bump_save_dir, 'bump_noise', 'progress_report.txt')
    trial_paths_path = os.path.join(bump_save_dir, 'bump_noise', 'trial_paths.txt')
else:
    results_save_dir = os.path.join(bump_save_dir, 'results')
    ensure_dir(results_save_dir)
    progress_report_path = os.path.join(bump_save_dir, 'progress_report.txt')
    trial_paths_path = os.path.join(bump_save_dir, 'trial_paths.txt')

print("Performing bump{} edits for {} class".format("-noise" if noise_edit else "", target_class_name))
print("Trial paths stored at {}".format(trial_paths_path))
print("Results will be saved to {}".format(results_save_dir))
# Remove file at trial_paths_path if it exists
if os.path.isfile(trial_paths_path):
    os.remove(trial_paths_path)

print(noise_edit)
if not noise_edit:
    trial_paths = replicate_bump_results(
        bumps_preds_metrics=bumps_preds_metrics,
        target_class_distribution=target_class_distribution,
        results_save_dir=results_save_dir,
        trial_paths_path=trial_paths_path)
else:
    # Create editor
    editor_args = config.config['editor']['args']
    editor_args['arch'] = config.config['arch']['args']['type']
    editor = Editor(**editor_args)
    # Recreate trial results with bump and noise
    trial_paths = replicate_bump_noise_results(
        bumps_preds_metrics=bumps_preds_metrics,
        target_class_distribution=target_class_distribution,
        results_save_dir=results_save_dir,
        trial_paths_path=trial_paths_path,
        progress_report_path=progress_report_path,
        # Parameters for adding noise
        # noise_edit=noise_edit,
        target_class_idx=target_class_idx,
        config=config,
        data_loader=val_paths_data_loader,
        loss_fn=loss_fn,
        metrics_fns=metric_fns,
        editor=editor,
        device=device)

        
    
print(trial_paths)
write_lists(trial_paths_path, trial_paths)
print("Wrote list of trial_paths to {}".format(trial_paths_path))

Performing bump-noise edits for airplane class
Trial paths stored at saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/trial_paths.txt
Results will be saved to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results
True
Bin 0 [7014.0, 7533.62)
	Itr 1/3
L2 norm of weight change: 4.605295658111572
pre_edit_weights id: 139767148232848 mean: 1.063853324012598e-05
post_edit_weights id: 139767144758304 mean: 1.1618755706876982e-05
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [05:06<00:00,  1.12s/it]


	Itr 2/3
L2 norm of weight change: 4.608025550842285
pre_edit_weights id: 139767148688864 mean: 1.063853324012598e-05
post_edit_weights id: 139767144073552 mean: 8.5252804638003e-06
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 29.02it/s]


	Itr 3/3
L2 norm of weight change: 4.610196590423584
pre_edit_weights id: 139767230537296 mean: 1.063853324012598e-05
post_edit_weights id: 139767144108496 mean: 1.4323152754514012e-05
are the tensors equal: False


...

100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:09<00:00, 30.25it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


Bin 49 [32475.38, 32995.0)
	Itr 1/1
L2 norm of weight change: 4.606288909912109
pre_edit_weights id: 139767232476560 mean: 1.063853324012598e-05
post_edit_weights id: 139767231556560 mean: 1.4464273590419907e-05
are the tensors equal: False


100%|█████████████████████████████████████████████████████████████████████| 274/274 [00:08<00:00, 34.18it/s]

['saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_0', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_1', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7014.0_7533.62_itr_2', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_0', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_1', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_2', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/bump_noise/results/bin_7533.62_8053.24_itr_3', 'saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/airplane_100/b


/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


In [34]:
# Save class distribution as histogram
save_class_distribution(
    csv_path='saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/automobile_100/bump_noise/results_table.csv', 
    target_class_idx=target_class_idx,
    target_class_name=target_class_name)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


--- Logging error ---
Traceback (most recent call last):
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/logging/handlers.py", line 69, in emit
    if self.shouldRollover(record):
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/logging/handlers.py", line 186, in shouldRollover
    self.stream.seek(0, 2)  #due to non-posix-compliant Windows feature
OSError: [Errno 116] Stale file handle
Call stack:
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/n/fs/ac-project/anaconda3/envs/editing/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/u/ac4802/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/u/ac4802/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/u/ac4802/.local/lib/python

Saved target class distribution & histogram data to saved/edit/experiments/bump_edits/CINIC10_ImageNet-VGG_16/0208_112555/automobile_100/bump_noise/target_class_distribution.pth


{'n_target_predictions': array([ 7828,  7820,  7847,  7778,  7828,  7813,  7824,  8694,  8699,
         8536,  8766,  8791,  8693,  8755,  8695,  8719,  8683,  8655,
         8736,  8644,  9548,  9500,  9308,  9414,  9442,  9516,  9459,
         9490,  9333,  9442,  9494,  9379,  9376,  9303, 10639, 10683,
        10639, 10705, 10822, 10765, 10866, 10790, 10773, 10598, 11074,
        11357, 11458, 11334, 11997, 11706, 13292, 14293, 14461, 14302,
        14142, 13842, 14053, 14068, 14090, 14571, 14658, 15060, 14843,
        14808, 14517, 14717, 14787, 14841, 16025, 15994, 15976, 15963,
        15965, 15987, 16293, 15814, 16077, 16264, 16253, 16328, 15907,
        16148, 16228, 16013, 16183, 16120, 17448, 16606, 16720, 16972,
        17111, 16866, 16381, 16661, 16841, 17077, 17237, 17583, 17898,
        17778, 17948, 17969, 17596, 17670, 17580, 18727, 18502, 18559,
        19167, 19248, 19225, 20326, 22684, 23171, 24276, 23299, 23760,
        25793, 28990, 28034, 31953, 53699]),
 'histog